# 🧩 13 — DIM_MODALIDADE (Gold)

- Lê `silver/2018_anonimizado.xlsx` e `silver/2019_anonimizado.xlsx`
- Normaliza modalidades
- Exporta em `gold/output/dim_modalidade.csv`


## 0) Imports

In [ ]:
import pandas as pd
import numpy as np


## 1) Paths robustos

In [ ]:
from pathlib import Path

# ======================================================
# Paths robustos (funciona no VS Code / Jupyter / OneDrive)
# - encontra a raiz do projeto procurando a pasta 'silver'
# ======================================================
CWD = Path().resolve()

def find_project_root(start: Path) -> Path:
    p = start
    for _ in range(12):
        if (p / "silver").exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    raise FileNotFoundError("Não encontrei a pasta 'silver' subindo a árvore. Rode o notebook dentro do repo.")

PROJECT_ROOT = find_project_root(CWD)
SILVER_DIR = PROJECT_ROOT / "silver"
GOLD_DIR = PROJECT_ROOT / "gold"
OUT_DIR = GOLD_DIR / "output"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("📌 CWD:", CWD)
print("📌 PROJECT_ROOT:", PROJECT_ROOT)
print("📌 SILVER_DIR:", SILVER_DIR)
print("📌 OUT_DIR:", OUT_DIR)


## 2) Ler Silver (2018 + 2019)

In [ ]:
import pandas as pd
import numpy as np

INPUT_FILES = [
    SILVER_DIR / "2018_anonimizado.xlsx",
    SILVER_DIR / "2019_anonimizado.xlsx",
]

print("📥 INPUT_FILES:")
for f in INPUT_FILES:
    print(" -", f, "| existe?", f.exists())

dfs = []
for f in INPUT_FILES:
    if not f.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {f}")
    tmp = pd.read_excel(f, dtype=str)
    tmp["fonte_arquivo"] = f.name
    dfs.append(tmp)

df = pd.concat(dfs, ignore_index=True)
print("✅ Linhas/Colunas consolidadas:", df.shape)
df.head()


## 3) Funções auxiliares

In [ ]:
import pandas as pd
import numpy as np

def norm_missing(s: pd.Series) -> pd.Series:
    x = s.astype(str).str.strip()
    return x.replace({"": np.nan, "nan": np.nan, "NAN": np.nan, "None": np.nan, "NONE": np.nan})

def dedup_most_complete(df_in: pd.DataFrame, key: str) -> pd.DataFrame:
    score = df_in.notna().sum(axis=1)
    return (df_in.assign(_score=score)
              .sort_values([key, "_score"], ascending=[True, False])
              .drop_duplicates(subset=[key], keep="first")
              .drop(columns=["_score"]))

def pick_first_existing(candidates, df_):
    return next((c for c in candidates if c in df_.columns), None)


## 4) Construir DIM_MODALIDADE

In [ ]:
if "MODALIDADE" not in df.columns:
    raise KeyError("Coluna MODALIDADE não encontrada.")

m = norm_missing(df["MODALIDADE"]).str.upper()

m_norm = (m.replace({
    "EAD": "EAD",
    "PRESENCIAL": "PRESENCIAL",
    "SEMI-PRESENCIAL": "SEMIPRESENCIAL",
    "SEMIPRESENCIAL": "SEMIPRESENCIAL",
})
.fillna("NÃO INFORMADO"))

dim_modalidade = (
    pd.DataFrame({"modalidade_norm": m_norm})
      .drop_duplicates()
      .sort_values("modalidade_norm")
      .reset_index(drop=True)
)

dim_modalidade["id_modalidade"] = dim_modalidade["modalidade_norm"].factorize()[0] + 1
dim_modalidade = dim_modalidade[["id_modalidade","modalidade_norm"]]

print("✅ DIM_MODALIDADE pronta:", dim_modalidade.shape)
dim_modalidade


## 5) Exportar

In [ ]:
out_file = OUT_DIR / "dim_modalidade.csv"
dim_modalidade.to_csv(out_file, index=False, encoding="utf-8")
print("✅ Salvo em:", out_file)
